In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import time, datetime
from arcticdb import Arctic, QueryBuilder
import yfinance as yf
import talib
# from arcticdb import Arctic, QueryBuilder
from utils import ac

lib = ac.get_library('stocks', create_if_missing=True)

symbol_df = pd.read_csv("us_stocks/symbols.csv",delimiter=',',index_col='Symbol').drop('Unnamed: 0',axis=1)
# Keep the first occurrence of each symbol
symbol_df = symbol_df[~symbol_df.index.duplicated(keep='first')]

symbols = symbol_df.index.unique().to_list()

Creating library 'general' where *settings and *strategies will be stored


In [10]:
symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA

In [45]:
data = yf.download(symbols[:500], group_by="Ticker", period="2y", auto_adjust=True)
df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)
df = df.sort_values(by='Symbol',axis='index',kind='stable').drop(axis=1,columns="Adj Close")

df["Name"] = df["Symbol"].map(symbol_df["Name"])
df['Sector'] = df['Symbol'].map(symbol_df['Sector'])
df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).reset_index(level=0, drop=True)
df
# print(df)
# # df["50D_SMA"] = df.groupby("Symbol")["Adj Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
# # df["200D_SMA"] = df.groupby("Symbol")["Adj Close"].rolling(window=200).mean().reset_index(level=0, drop=True)
#df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20).stack()).reset_index(level=0, drop=True)


[*********************100%%**********************]  500 of 500 completed


2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=2y)')
['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')


,Symbol,Open,High,Low,Close,Volume,Name,Sector,ATR
Date,,,,,,,,,
2022-01-10,A,141.326528,143.318852,138.929832,143.170914,2548100.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-11,A,143.013088,144.926507,141.839392,144.630615,2194200.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-12,A,145.774744,148.329250,145.528169,147.461304,2250800.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-13,A,147.412010,147.490901,142.865179,143.180786,1741800.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-14,A,142.066250,143.161040,140.409277,142.697479,2225400.0,"Agilent Technologies, Inc.",Health Technology,NaN
...,...,...,...,...,...,...,...,...,...
2024-01-02,ZBRA,268.559998,270.739990,265.390015,267.980011,452900.0,Zebra Technologies Corporation,Electronic Technology,6.872673
2024-01-03,ZBRA,264.119995,265.549988,252.380005,252.520004,404800.0,Zebra Technologies Corporation,Electronic Technology,7.309039
2024-01-04,ZBRA,251.100006,255.410004,250.529999,252.970001,358100.0,Zebra Technologies Corporation,Electronic Technology,7.187587


In [44]:
df["Name"] = df["Symbol"].map(symbol_df["Name"])
df['Sector'] = df['Symbol'].map(symbol_df['Sector'])
df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).reset_index(level=0, drop=True)

df

,Symbol,Open,High,Low,Close,Volume,Name,Sector,ATR
Date,,,,,,,,,
2022-01-10,A,141.326513,143.318837,138.929818,143.170898,2548100.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-11,A,143.013088,144.926507,141.839392,144.630615,2194200.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-12,A,145.774729,148.329235,145.528154,147.461288,2250800.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-13,A,147.411995,147.490885,142.865164,143.180771,1741800.0,"Agilent Technologies, Inc.",Health Technology,NaN
2022-01-14,A,142.066234,143.161025,140.409262,142.697464,2225400.0,"Agilent Technologies, Inc.",Health Technology,NaN
...,...,...,...,...,...,...,...,...,...
2024-01-02,ZBRA,268.559998,270.739990,265.390015,267.980011,452900.0,Zebra Technologies Corporation,Electronic Technology,6.872673
2024-01-03,ZBRA,264.119995,265.549988,252.380005,252.520004,404800.0,Zebra Technologies Corporation,Electronic Technology,7.309039
2024-01-04,ZBRA,251.100006,255.410004,250.529999,252.970001,358100.0,Zebra Technologies Corporation,Electronic Technology,7.187587


In [39]:
# Calculate ATR for each group
atr_df = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20))

# Reshape the DataFrame into long format
atr_df = atr_df.reset_index().melt(id_vars='Symbol', var_name='Date', value_name='ATR')
atr_df

,Symbol,Date,ATR
0,ADP,2022-01-10 00:00:00,NaN
1,AVB,2022-01-10 00:00:00,NaN
2,AVGO,2022-01-10 00:00:00,NaN
3,AVY,2022-01-10 00:00:00,NaN
4,AXON,2022-01-10 00:00:00,NaN
...,...,...,...
24043,CPT,2024-01-08 00:00:00,2.068218
24044,CTLT,2024-01-08 00:00:00,1.631359
24045,CZR,2024-01-08 00:00:00,1.824953
24046,KMX,2024-01-08 00:00:00,2.544061


In [32]:
print(df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).head())


Date    2022-01-10  2022-01-11  2022-01-12  2022-01-13  2022-01-14  \
Symbol                                                               
ADP            NaN         NaN         NaN         NaN         NaN   
AVB            NaN         NaN         NaN         NaN         NaN   
AVGO           NaN         NaN         NaN         NaN         NaN   
AVY            NaN         NaN         NaN         NaN         NaN   
AXON           NaN         NaN         NaN         NaN         NaN   

Date    2022-01-18  2022-01-19  2022-01-20  2022-01-21  2022-01-24  ...  \
Symbol                                                              ...   
ADP            NaN         NaN         NaN         NaN         NaN  ...   
AVB            NaN         NaN         NaN         NaN         NaN  ...   
AVGO           NaN         NaN         NaN         NaN         NaN  ...   
AVY            NaN         NaN         NaN         NaN         NaN  ...   
AXON           NaN         NaN         NaN         NaN     

In [3]:
def split_symbols(symbols, chunk_size):
    """Split a list of symbols into smaller chunks."""
    for i in range(0, len(symbols), chunk_size):
        yield symbols[i:i + chunk_size]

# Function to download and store data
def download_and_store(chunk):
    data = yf.download(chunk, group_by="Ticker", period="3y", auto_adjust=True)

    df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)
    df = df.sort_values(by='Symbol',axis='index',kind='stable')
    print(df)
    # Add additional information
    df["Name"] = df["Symbol"].map(symbol_df["Name"])
    df['Sector'] = df['Symbol'].map(symbol_df['Sector'])

    df["20D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=20).mean().reset_index(level=0, drop=True)
    df["50D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
    df["200D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)
    # df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=14)).reset_index(level=0, drop=True)
    print(df)
    def calculate_atr(group):
        print(group['Close'])
        return talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)

    df['ATR'] = df.groupby('Symbol').apply(calculate_atr).reset_index(level=0, drop=True)

    df['1M'] = df.groupby('Symbol')['Close'].pct_change(21)
    df['3M'] = df.groupby('Symbol')['Close'].pct_change(63)
    df['6M'] = df.groupby('Symbol')['Close'].pct_change(126)
    df['12M'] = df.groupby('Symbol')['Close'].pct_change(252)
    df['RS IBD'] = 2*df['3M']+df['6M']+df['12M'] # IBD Relative Strength =  2x 3M + 1x 6M + 1x 12M
    df['RS Rank'] = df.groupby(df.index)['RS IBD'].rank(pct=True)
    df["RS Rank 20D MA"] = df.groupby("Symbol")["RS Rank"].rolling(window=20).mean().reset_index(level=0, drop=True)

    for symbol in df.Symbol.unique().tolist():
        stock_data = df[df['Symbol']==symbol]
        # Store in Arctic
        print(f"saving {symbol} to arcticdb")
        lib.write(symbol, stock_data)



In [4]:
# Example usage
chunked_symbols = list(split_symbols(symbols, 2))
print(f"Splitting symbols into {len(chunked_symbols)} chunks.")

Splitting symbols into 2302 chunks.


In [5]:

# Download and store data for each chunk
for chunk in chunked_symbols[:2]:
    print(chunk)
    download_and_store(chunk)

['A', 'AA']
[*********************100%%**********************]  2 of 2 completed
           Symbol        Open        High         Low       Close   Volume
Date                                                                      
2021-01-11      A  124.046285  126.881348  123.830473  125.713966  1746800
2021-01-12      A  125.890537  125.949394  123.801022  124.556389  1826400
2021-01-13      A  125.037095  125.615883  124.330780  124.566223  1530400
2021-01-14      A  124.850696  125.753213  123.428263  123.634270   924900
2021-01-15      A  123.545988  124.703557  122.368794  123.810852  2115400
...           ...         ...         ...         ...         ...      ...
2024-01-02     AA   33.529999   33.970001   33.040001   33.310001  3838000
2024-01-03     AA   32.119999   32.270000   31.045000   31.490000  8245700
2024-01-04     AA   31.129999   31.490000   30.860001   31.129999  4543600
2024-01-05     AA   30.540001   32.305000   30.090000   32.130001  7313000
2024-01-08     AA  

ValueError: Cannot set a DataFrame with multiple columns to the single column ATR

In [28]:
spx500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",index_col=0)[0]
symbols = spx500.index.to_list()
data = yf.download(symbols, group_by="Ticker", period="2y", auto_adjust=True)


[*********************100%%**********************]  503 of 503 completed


2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=2y)')
['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')


In [29]:
df = data.stack(level=0).rename_axis(['Date', 'Symbol']).reset_index(level=1)
df = df.sort_values(by='Symbol',axis='index',kind='stable').drop(axis=1,columns="Adj Close")

In [30]:
# Map the sectors and industries to the df DataFrame
df["Name"] = df["Symbol"].map(spx500["Security"])
df['Sector'] = df['Symbol'].map(spx500['GICS Sector'])
df['Industry'] = df['Symbol'].map(spx500["GICS Sub-Industry"])
df
# df["50D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
# df["200D_SMA"] = df.groupby("Symbol")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)
# df['ATR'] = df.groupby('Symbol').apply(lambda group: talib.ATR(group['High'], group['Low'], group['Close'], timeperiod=20)).reset_index(level=0, drop=True)


,Symbol,Open,High,Low,Close,Volume,Name,Sector,Industry
Date,,,,,,,,,
2022-01-10,A,141.326528,143.318852,138.929832,143.170914,2548100.0,Agilent Technologies,Health Care,Life Sciences Tools & Services
2022-01-11,A,143.013104,144.926523,141.839407,144.630630,2194200.0,Agilent Technologies,Health Care,Life Sciences Tools & Services
2022-01-12,A,145.774729,148.329235,145.528154,147.461288,2250800.0,Agilent Technologies,Health Care,Life Sciences Tools & Services
2022-01-13,A,147.411979,147.490869,142.865148,143.180756,1741800.0,Agilent Technologies,Health Care,Life Sciences Tools & Services
2022-01-14,A,142.066250,143.161040,140.409277,142.697479,2225400.0,Agilent Technologies,Health Care,Life Sciences Tools & Services
...,...,...,...,...,...,...,...,...,...
2024-01-02,ZTS,195.789993,197.949997,195.050003,196.570007,1642300.0,Zoetis,Health Care,Pharmaceuticals
2024-01-03,ZTS,195.919998,195.949997,192.800003,192.929993,1493000.0,Zoetis,Health Care,Pharmaceuticals
2024-01-04,ZTS,192.850006,194.929993,192.009995,194.039993,1851900.0,Zoetis,Health Care,Pharmaceuticals


In [9]:
df

,Symbol,Open,High,Low,Close,Volume,Name,Sector,Industry,50D_SMA,200D_SMA,ATR
Date,,,,,,,,,,,,
2022-01-10,A,141.326513,143.318837,138.929818,143.170898,2548100.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN
2022-01-11,A,143.013104,144.926523,141.839407,144.630630,2194200.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN
2022-01-12,A,145.774729,148.329235,145.528154,147.461288,2250800.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN
2022-01-13,A,147.411995,147.490885,142.865164,143.180771,1741800.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN
2022-01-14,A,142.066234,143.161025,140.409262,142.697464,2225400.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-02,ZTS,195.789993,197.949997,195.050003,196.570007,1642300.0,Zoetis,Health Care,Pharmaceuticals,178.442557,176.043526,3.441222
2024-01-03,ZTS,195.919998,195.949997,192.800003,192.929993,1493000.0,Zoetis,Health Care,Pharmaceuticals,178.967339,176.191344,3.457661
2024-01-04,ZTS,192.850006,194.929993,192.009995,194.039993,1851900.0,Zoetis,Health Care,Pharmaceuticals,179.526093,176.338008,3.430778
